In [12]:
import pandas as pd
from keras.datasets import mnist
from keras.layers import Input, Dense, Reshape, Flatten, Dropout, multiply
from keras.layers import BatchNormalization, Activation, Embedding, ZeroPadding2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.convolutional import UpSampling2D, Conv2D
from keras.models import Sequential, Model
import numpy as np
from sklearn.model_selection import  train_test_split

In [2]:
data=pd.read_excel("data.xlsx")
data

,Unnamed: 0,features,class
0,0,[[[ 16 104 24]\n [ 15 105 20]\n [ 15 107 ...,septoria
1,1,[[[ 1 133 52]\n [ 1 130 46]\n [ 1 124 ...,septoria
2,2,[[[ 0 0 0]\n [ 0 0 0]\n [ 0 0 ...,septoria
3,3,[[[ 0 0 0]\n [ 0 0 0]\n [ 0 0 ...,septoria
4,4,[[[ 18 194 151]\n [ 15 194 159]\n [ 27 213 1...,septoria
...,...,...,...
1193,1193,[[[ 90 193 148]\n [ 85 188 143]\n [ 81 184 1...,healthy
1194,1194,[[[ 53 73 74]\n [ 43 63 64]\n [ 33 51 ...,healthy
1195,1195,[[[ 58 81 89]\n [ 43 66 74]\n [ 32 55 ...,healthy
1196,1196,[[[ 7 72 50]\n [ 3 68 46]\n [ 5 65 ...,healthy


In [48]:
x=features=data["features"]
y=class_data=data["class"]
features
x.shape

(1198,)

In [9]:
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.30,random_state=42)


(838,)

In [49]:

def generator(): 
    model = Sequential()

    model.add(Dense(128 * 7 * 7, activation="relu", input_dim=100))
    model.add(Reshape((7, 7, 128)))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(128, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(UpSampling2D())
    model.add(Conv2D(64, kernel_size=3, padding="same"))
    model.add(Activation("relu"))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(1, kernel_size=3, padding='same'))
    model.add(Activation("tanh"))
    model.summary()
    noise = Input(shape=(100,))
    label = Input(shape=(1,), dtype='int32')
    label_embedding = Flatten()(Embedding(2, 100)(label))
    model_input = multiply([noise, label_embedding])
    img = model(model_input)
    return Model([noise, label], img)

In [50]:
   
def discriminator():

    model = Sequential()

    model.add(Conv2D(16, kernel_size=3, strides=2, input_shape=x_train.shape, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(Conv2D(32, kernel_size=3, strides=2, padding="same"))
    model.add(ZeroPadding2D(padding=((0,1),(0,1))))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(64, kernel_size=3, strides=2, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))
    model.add(BatchNormalization(momentum=0.8))
    model.add(Conv2D(128, kernel_size=3, strides=1, padding="same"))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.25))

    model.add(Flatten())
    model.summary()

    img = Input(shape=x_train.shape)

    # Extract feature representation
    features = model(img)

    # Determine validity and label of the image
    validity = Dense(1, activation="sigmoid")(features)
    label = Dense(3, activation="softmax")(features)
    return Model(img, [validity, label])

In [51]:
dis=discriminator()
dis.compile(loss="binary_crossentropy",optimizer="Adam",metrics=['accuracy'])

ValueError: Input 0 of layer "conv2d_33" is incompatible with the layer: expected min_ndim=4, found ndim=2. Full shape received: (None, 838)

In [ ]:
gen=generator()

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_14 (Dense)            (None, 6272)              633472    
                                                                 
 reshape_3 (Reshape)         (None, 7, 7, 128)         0         
                                                                 
 batch_normalization_19 (Bat  (None, 7, 7, 128)        512       
 chNormalization)                                                
                                                                 
 up_sampling2d_6 (UpSampling  (None, 14, 14, 128)      0         
 2D)                                                             
                                                                 
 conv2d_30 (Conv2D)          (None, 14, 14, 128)       147584    
                                                                 
 activation_9 (Activation)   (None, 14, 14, 128)     

In [30]:
noise = Input(shape=(100,))
label = Input(shape=(1,))
img =gen([noise, label])
valid, target_label =dis(img)


In [31]:
combined = Model([noise, label], [valid, target_label])
combined.compile(loss="binary_crossentropy",optimizer="Adam")


In [45]:
print(y_train[100])

KeyError: 100

In [47]:
# Load the datase

# Adversarial ground truths
batch_size=100
epochs=20
valid = np.ones((batch_size, 1))
fake = np.zeros((batch_size, 1))

for epoch in range(epochs):

# ---------------------
#  Train Discriminator
# ---------------------

# Select a random batch of images
    idx = np.random.randint(0, x_train.shape[0], batch_size)
    imgs = x_train.values[idx]

    # Sample noise as generator input
    noise = np.random.normal(0, 1, (batch_size, 100))

    # The labels of the digits that the generator tries to create an
    # image representation of
    sampled_labels = np.random.randint(0, 10, (batch_size, 1))

# Generate a half batch of new images
    gen_imgs =gen.predict([noise, sampled_labels])

    # Image labels. 0-9 if image is valid or 10 if it is generated (fake)
    img_labels = y_train.values[idx]
    fake_labels = 10 * np.ones(img_labels.shape)

    # Train the discriminator
    d_loss_real = dis.train_on_batch(imgs, [valid, img_labels])
    d_loss_fake = dis.train_on_batch(gen_imgs, [fake, fake_labels])
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

# ---------------------
#  Train Generator
# ---------------------

# Train the generator
    g_loss = combined.train_on_batch([noise, sampled_labels], [valid, sampled_labels])

    # Plot the progress
    #             print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))

    # If at save interval => save generated image samples
    '''
    if epoch % 50 == 0:
        save_model()
        sample_images(epoch)
        print ("%d [D loss: %f, acc.: %.2f%%, op_acc: %.2f%%] [G loss: %f]" % (epoch, d_loss[0], 100*d_loss[3], 100*d_loss[4], g_loss[0]))
'''

InvalidArgumentError: Graph execution error:

Detected at node 'model_5/embedding_3/embedding_lookup' defined at (most recent call last):
    File "/usr/lib64/python3.10/runpy.py", line 196, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/usr/lib64/python3.10/runpy.py", line 86, in _run_code
      exec(code, run_globals)
    File "/home/hrithikpaul/.local/lib/python3.10/site-packages/ipykernel_launcher.py", line 17, in <module>
      app.launch_new_instance()
    File "/home/hrithikpaul/.local/lib/python3.10/site-packages/traitlets/config/application.py", line 965, in launch_instance
      app.start()
    File "/home/hrithikpaul/.local/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 712, in start
      self.io_loop.start()
    File "/home/hrithikpaul/.local/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 199, in start
      self.asyncio_loop.run_forever()
    File "/usr/lib64/python3.10/asyncio/base_events.py", line 600, in 